In [25]:
import polars as pl
import pandas as pd
import numpy as np
import os, glob
import re
import plotly.express as px
import polars as pl

In [107]:
# directories
cwd = os.getcwd() # directory this file is saved in
data_dir = os.path.join(cwd, "BCDC-Metadata") # BCDC data folder
metadata_dir = os.path.join(data_dir, 'Sample-Inventory') # directory with specimen metadata
project_dir = os.path.join(data_dir, 'Data-Collection-Inventory', '2022Q3_csvs')

df = pd.read_csv(os.path.join(cwd, 'BCDC_Metadata_2022Q3_v4.csv'), encoding='unicode_escape')

### use a data cleaning function here that takes both this dataset and CV
# 1. get rid of all the special characters in CV columns and column names
# 2. cv and data columns both to lower case
# 3. way to identify extremely similar words (i.e. brain vs brains)
### in simple code logic...
# list of cols with controlled values
controlled_cols = [re.sub("\([^)]*\)", "", x).strip() for x in list(filter(lambda x : 'CV' in x, df.columns))]
df.columns = df.columns.str.replace(r"\([^)]*\)","").str.strip() # get rid of (CV) in data columns
df['Sample Type'] = df['Sample Type'].str.replace("[^A-Za-z0-9 ]+", " ").str.lower() # special characters into spaces
# to-do: do the same for all CV columns
df['Subspecimen Type'] = df['Subspecimen Type'].str.replace("[^A-Za-z0-9 ]+", " ").str.lower() # special characters into spaces
df['Total Processed Subspecimens'] = df['Total Processed Subspecimens'].astype(str).str.replace(',', '').astype(float) # cell counts is numeric
df['Sample ID'] = df['Sample ID'].astype(str)
df['Species'] = df['Species'].str.lower()

df['Sample Type'] = df['Sample Type'].fillna(df['Subspecimen Type'])
df['Subspecimen Type'] = df['Subspecimen Type'].fillna(df['Sample Type'])
df['Total Processed Subspecimens'] = df['Total Processed Subspecimens'].fillna(0)
### collect systematic patterns for auto-correction
optional_cols = ['Age', 'Sex', 'Genotype', 'Species NCBI Taxonomy ID']
df = df.drop(optional_cols, axis = 1)
df = df.drop_duplicates()

df.loc[(df['Technique'].str.contains('mc-seq', case = False)) & (~df['Technique'].str.contains(';', case = False)), 'Technique'] = 'mC-seq2'
df

/var/folders/76/jdv5bsqj4_9dk0rm_g_vp9gm0000gp/T/ipykernel_76632/4265238037.py:7: DtypeWarning: Columns (4,5,6,7,8,9,12,14,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(cwd, 'BCDC_Metadata_2022Q3_v4.csv'), encoding='unicode_escape')
/var/folders/76/jdv5bsqj4_9dk0rm_g_vp9gm0000gp/T/ipykernel_76632/4265238037.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(r"\([^)]*\)","").str.strip() # get rid of (CV) in data columns
/var/folders/76/jdv5bsqj4_9dk0rm_g_vp9gm0000gp/T/ipykernel_76632/4265238037.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Sample Type'] = df['Sample Type'].str.replace("[^A-Za-z0-9 ]+", " ").str.lower() # special characters into spaces
/var/folders/76/jdv5bsqj4_9dk0rm_g_vp9gm0000gp/T/ipykernel_76632/4265238037.py:19: FutureWarning: The default value of regex 

,Sample ID,Sample Type,Species,Parent Specimen ID,Parent Specimen Type,Subject ID,Modality,Technique,Anatomical Structure,Subspecimen Type,Total Processed Subspecimens,Organization,Investigator,Grant Number,Data Collection,R24 Name,R24 link,Comments,Metadata Submission
0,SW170829-01A,whole brain,mouse,NaN,NaN,SW170829-01A,connectivity,anterograde tracing,NaN,whole brain,1.0,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3
1,SW170829-02A,whole brain,mouse,NaN,NaN,SW170829-02A,connectivity,anterograde tracing,NaN,whole brain,1.0,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3
2,SW170910-01A,whole brain,mouse,NaN,NaN,SW170910-01A,connectivity,anterograde tracing,NaN,whole brain,1.0,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3
3,SW170917-01A,whole brain,mouse,NaN,NaN,SW170917-01A,connectivity,anterograde tracing,NaN,whole brain,1.0,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3
4,SW171101-01A,whole brain,mouse,NaN,NaN,SW171101-01A,connectivity,anterograde tracing,NaN,whole brain,1.0,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155656,1138845290,cell in slice,human,NaN,NaN,H21.28.030,cell morphology,Patch-seq,TemL,cell in slice,1.0,Allen Institute for Brain Science,Ed Lein,1U01MH114812-10,lein_human_pseq_m,BIL,/bil/lz/rdaibs/efb9b12ba2fab63d/1138845290,https://download.brainimagelibrary.org/ef/b9/e...,2022Q3
155657,1126880187,cell in slice,human,NaN,NaN,H21.28.031,cell morphology,Patch-seq,IFG,cell in slice,1.0,Allen Institute for Brain Science,Ed Lein,1U01MH114812-11,lein_human_pseq_m,BIL,/bil/lz/rdaibs/efb9b12ba2fab63d/1126880187,https://download.brainimagelibrary.org/ef/b9/e...,2022Q3
155658,1126880246,cell in slice,human,NaN,NaN,H21.29.201,cell morphology,Patch-seq,IFG,cell in slice,1.0,Allen Institute for Brain Science,Ed Lein,1U01MH114812-12,lein_human_pseq_m,BIL,/bil/lz/rdaibs/efb9b12ba2fab63d/1126880246,https://download.brainimagelibrary.org/ef/b9/e...,2022Q3
155659,1159995431,cell in slice,human,NaN,NaN,H22.03.301,cell morphology,Patch-seq,TemL,cell in slice,1.0,Allen Institute for Brain Science,Ed Lein,1U01MH114812-13,lein_human_pseq_m,BIL,/bil/lz/rdaibs/efb9b12ba2fab63d/1159995431,https://download.brainimagelibrary.org/ef/b9/e...,2022Q3


#### Count for number of samples

In [126]:
## counts on sample type level
sample_count_df = pd.DataFrame(df.groupby(['Data Collection', 'Metadata Submission', 'Species'], as_index = False).agg({'Sample ID': 'nunique'})).rename(columns = {'Data Collection':'data_collection_reference_id', 'Sample ID':'sample_count', 'Metadata Submission':'quarters', 'Sample Type': 'sample_name', 'Species':'species'})
## count the unique instances of sample IDs and subject IDs for each unique combination of project, quarter and sample type
sample_count_df

,data_collection_reference_id,quarters,species,sample_count
0,chung_lsfm,2021Q4,human,1
1,chung_lsfm,2022Q2,human,1
2,chung_lsfm,2022Q3,human,1
3,dev_mouse_brain_lsfm,2021Q3,mouse,117
4,dev_mouse_brain_lsfm,2021Q4,mouse,81
...,...,...,...,...
430,zhuang_dulac_merfish_MOE,2021Q1,mouse,1
431,zhuang_dulac_merfish_PAG,2020Q2,mouse,1
432,zhuang_dulac_merfish_PAG_2,2021Q2,mouse,1
433,zhuang_merfish_M1,2020Q1,mouse,1


In [127]:
species_map = {
    'primates' : ['chimpanzee', 'small-eared galago', 'western gorilla', 'green monkey', 'pig-tailed macaque',
                 "ma's night monkey", 'rhesus macaque', 'bolivian squirrel monkey',
       'crab-eating macaque'],
    'humans' : ['human'], ## mostly mice and humans
    'mice' : ['mouse'],
    'marmoset': ['marmoset'],
    'small mammals': [
    'arctic ground squirrel',
    'nine-banded armadillo',
    'domestic cat',
    'domestic ferret',
    'gray short-tailed opossum',
    'pig', 'rabbit', 'norway rat', 'common tree shrew'
    ]
}


sample_count_df['species'] = sample_count_df['species'].apply(lambda i:[k for k, v in species_map.items() if i in v]).str[0] # should just have 1 for now
# calculate new sum in collapsed group
sample_count_df = sample_count_df.groupby(['data_collection_reference_id', 'quarters', 'species'], as_index = False)['sample_count'].sum()

,data_collection_reference_id,quarters,species,sample_count
0,chung_lsfm,2021Q4,humans,1
1,chung_lsfm,2022Q2,humans,1
2,chung_lsfm,2022Q3,humans,1
3,dev_mouse_brain_lsfm,2021Q3,mice,117
4,dev_mouse_brain_lsfm,2021Q4,mice,81
...,...,...,...,...
427,zhuang_dulac_merfish_MOE,2021Q1,mice,1
428,zhuang_dulac_merfish_PAG,2020Q2,mice,1
429,zhuang_dulac_merfish_PAG_2,2021Q2,mice,1
430,zhuang_merfish_M1,2020Q1,mice,1


In [128]:
sample_count_df.to_csv(os.path.join(cwd, 'dash_sample_count_df.csv'), index = False)

#### Count of subspecimens

In [36]:
## get number of donors per quarter
df_donors = df.groupby(['Data Collection', 'Metadata Submission'], as_index = False).agg({'Subject ID':'nunique'}).rename(columns = {'Data Collection':'data_collection_reference_id', 'Metadata Submission': 'quarters', 'Subject ID': 'donor_count'})
## get number of processed subspeciemns for each unique sample
df_subspecimens = df.groupby(['Data Collection', 'Metadata Submission', 'Sample ID', 'Subspecimen Type'], as_index = False).agg({'Total Processed Subspecimens':'sum'}).drop('Sample ID', axis = 1).rename(columns = {'Data Collection':'data_collection_reference_id', 'Metadata Submission': 'quarters', 'Subspecimen Type': 'subspecimen_type', 'Total Processed Subspecimens': 'subspecimen_count'})

df_donor_counts = df_donors.merge(df_subspecimens, on = ['data_collection_reference_id', 'quarters'])
# grant/data collection lookup table
key_df = pd.read_csv(os.path.join(project_dir, 'data_collection_is_specified_output_of_data_collection_project.csv')).drop('priority_order', axis = 1)
# grant/data collection lookup table
grant_df = pd.read_csv(os.path.join(project_dir, 'grant_is_specified_input_of_data_collection_project.csv')).drop('priority_order', axis = 1)
metadata_df = grant_df.merge(key_df, on = 'project_reference_id', how = 'outer')[['data_collection_reference_id', 'grant_reference_id']]
df_donor_counts = metadata_df.merge(df_donor_counts, on = 'data_collection_reference_id', how = 'right')


In [42]:
df_donor_counts

,data_collection_reference_id,grant_reference_id,quarters,donor_count,subspecimen_type,subspecimen_count
0,chung_lsfm,U01 Chung grant,2021Q4,0,brain section set,79.0
1,chung_lsfm,U01 Chung grant,2022Q2,0,brain section set,41.0
2,chung_lsfm,U01 Chung grant,2022Q3,0,brain section set,32.0
3,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q3,0,whole brain,1.0
4,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q3,0,whole brain,1.0
...,...,...,...,...,...,...
225284,zhuang_dulac_merfish_PAG,U19 Huang grant,2020Q2,0,whole cell,2900000.0
225285,zhuang_dulac_merfish_PAG_2,U19 Huang grant,2021Q2,0,whole cell,2900000.0
225286,zhuang_merfish_M1,multi-grant,2020Q1,0,whole cell,300000.0
225287,zhuang_merfish_M1,multi-grant,2020Q1,0,whole cell,300000.0


In [41]:
## projects funded by multiple grants
dup_df = df_donor_counts[['data_collection_reference_id', 'grant_reference_id']].drop_duplicates()
dup_count = pd.DataFrame(dup_df.drop_duplicates().groupby('data_collection_reference_id', as_index = False)['grant_reference_id'].nunique())
multi_list = dup_count[dup_count['grant_reference_id'] > 1]['data_collection_reference_id']
df_donor_counts.loc[df_donor_counts['data_collection_reference_id'].isin(multi_list), 'grant_reference_id'] = 'multi-grant'

In [43]:
specimen_count_map = {
'brain_count' : ['whole brain',
            'brain', 'brains'], # fix the plural
'cell_count' : ['whole cell', 
           'cell', 'cells', 'cell body', 'cell nucleus', 'nuclei', 'reconstruction', 'cell in slice'],
'nucleus_count' : ['cell nucleus', 'nuclei'],
'library_count' : ['library'],
'tissue_region_count' : ['brain hemisphere', 
                    'brain region', 
                    'brain section set', 
                    'brain slice', 
                    'library', 
                    'spinal cord slice',
                    'tissue sample',
                    'cell suspension']
}
    

count_df = pd.DataFrame()

for project in df_donor_counts['data_collection_reference_id'].unique():
    non_count_cols = ['data_collection_reference_id', 'grant_reference_id', 'quarters', 'donor_count', 'subspecimen_type']
    count_df_project = pd.DataFrame(columns = [non_count_cols + list(specimen_count_map.keys())], index = [0])
    project_df = df_donor_counts[df_donor_counts['data_collection_reference_id'] == project]
    project_df = project_df.groupby(non_count_cols, as_index = False)['subspecimen_count'].sum()
   # print(project_df)
    ## each quarter
    for i in range(len(project_df)):
        project_row = project_df.iloc[i]
        row_container = pd.DataFrame(columns = [non_count_cols + list(specimen_count_map.keys())], index = [0])
        #row_container[non_count_cols] = project_row[non_count_cols].drop('subspecimen_type')
        row_container[['data_collection_reference_id', 'grant_reference_id', 'quarters', 'donor_count']] = project_row[['data_collection_reference_id', 'grant_reference_id', 'quarters', 'donor_count']]
        count_cols = [k for k, v in specimen_count_map.items() if project_row['subspecimen_type'] in v]
        row_container[count_cols] = project_row['subspecimen_count']
        count_df_project = pd.concat([count_df_project, row_container.fillna(0)])
    
    count_df_project = count_df_project.dropna(how = 'all')
    count_df = pd.concat([count_df, count_df_project])


count_df 

,data_collection_reference_id,grant_reference_id,quarters,donor_count,subspecimen_type,brain_count,cell_count,nucleus_count,library_count,tissue_region_count
0,chung_lsfm,U01 Chung grant,2021Q4,0,0,0,0,0,0,79.0
0,chung_lsfm,U01 Chung grant,2022Q2,0,0,0,0,0,0,41.0
0,chung_lsfm,U01 Chung grant,2022Q3,0,0,0,0,0,0,32.0
0,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q3,0,0,117.0,0,0,0,0
0,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q4,27,0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
0,zhuang_dulac_merfish_MOE,U19 Zeng grant,2021Q1,0,0,0,3300000.0,0,0,0
0,zhuang_dulac_merfish_PAG,U19 Huang grant,2020Q2,0,0,0,2900000.0,0,0,0
0,zhuang_dulac_merfish_PAG_2,U19 Huang grant,2021Q2,0,0,0,2900000.0,0,0,0
0,zhuang_merfish_M1,multi-grant,2020Q1,0,0,0,600000.0,0,0,0


In [44]:
count_df

,data_collection_reference_id,grant_reference_id,quarters,donor_count,subspecimen_type,brain_count,cell_count,nucleus_count,library_count,tissue_region_count
0,chung_lsfm,U01 Chung grant,2021Q4,0,0,0,0,0,0,79.0
0,chung_lsfm,U01 Chung grant,2022Q2,0,0,0,0,0,0,41.0
0,chung_lsfm,U01 Chung grant,2022Q3,0,0,0,0,0,0,32.0
0,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q3,0,0,117.0,0,0,0,0
0,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q4,27,0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
0,zhuang_dulac_merfish_MOE,U19 Zeng grant,2021Q1,0,0,0,3300000.0,0,0,0
0,zhuang_dulac_merfish_PAG,U19 Huang grant,2020Q2,0,0,0,2900000.0,0,0,0
0,zhuang_dulac_merfish_PAG_2,U19 Huang grant,2021Q2,0,0,0,2900000.0,0,0,0
0,zhuang_merfish_M1,multi-grant,2020Q1,0,0,0,600000.0,0,0,0


In [45]:
count_df.to_csv(os.path.join(cwd, 'dash_specimen_count_df.csv'), index = False)